In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.geometry import setup_geometry, shepp_logan
from src.plot import animate_S_D
from IPython.display import HTML

In [ ]:
N_pixels = 256
R_obj = 125
D_sd = 500
D_so = 250
N_det = 128
N_views = 100
view_range = 2 * np.pi
mode = "equiangular"

phantom = shepp_logan(N_pixels)

S, D, theta, beta, fan_angle, delta_beta = setup_geometry(
    D_so, D_sd, R_obj, N_det, N_views, view_range, mode
)

anim = animate_S_D(S, D, phantom)
HTML(anim.to_html5_video())

# sinogram = acquire_fanbeam_projections(phantom, S, D)

# filter = g(np.arange(-N_det / 2, N_det /2), delta_beta)

# sinogram_filt = convolve(sinogram, filter.reshape(1, -1), mode="same")

# fig, axs = plt.subplots(1,3)

# axs[0].imshow(sinogram, cmap="gray")
# axs[1].imshow(sinogram_filt, cmap="gray")
# axs[2].imshow(np.abs(sinogram - sinogram_filt), cmap="gray")

# plt.tight_layout()

In [ ]:
from src.project import acquire_fanbeam_projections
from src.filter import build_freq_filter, filter_multiply

# Acquire projections
sinogram = acquire_fanbeam_projections(phantom, S, D)

# Implement distance correction
sinogram_corr = sinogram * D_so * np.cos(beta[None, :])

# Filter the projections
filter = build_freq_filter(N_det, "shepp-logan", 0.5)
sinogram_filt = filter_multiply(sinogram_corr, filter, delta_beta)

fig, axs = plt.subplots(1, 3, figsize=(30, 10))

axs[0].imshow(sinogram, cmap="gray")
axs[0].set_title("Acquired sinogram")

axs[1].imshow(sinogram_corr, cmap="gray")
axs[1].set_title("Distance corrected sinogram")

axs[2].imshow(sinogram_filt, cmap="gray")
axs[2].set_title("Filtered sinogram")

In [ ]:
from src.reconstruct import compute_source_frame_coords

L, gamma = compute_source_frame_coords(N_pixels, D_so, theta)

In [ ]:
view = 74


# Get image dimensions
ny, nx = L[..., view].shape

# Define extent so that (0, 0) is at the center
extent = [-nx // 2, nx // 2, -ny // 2, ny // 2]

fig, axs = plt.subplots(1, 2, figsize=(20, 10))

# First subplot
im0 = axs[0].imshow(L[..., view], extent=extent)
axs[0].scatter(S[view, 0], S[view, 1], label="Source")
axs[0].scatter(D[view, :, 0], D[view, :, 1], label="Detector")
fig.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].legend()
axs[0].set_title("L Image")

# Second subplot
im1 = axs[1].imshow(gamma[..., view], extent=extent)
axs[1].scatter(S[view, 0], S[view, 1], label="Source")
axs[1].scatter(D[view, :, 0], D[view, :, 1], label="Detector")
fig.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].legend()
axs[1].set_title("Gamma Image")

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np

S_view = S[0]
D_view = D[0]

# Plotting
fig, ax = plt.subplots(figsize=(50, 50))
ax.plot(S_view[0], S_view[1], "ro", label="Source")  # plot source

# Plot detector points
ax.plot(D_view[:, 0], D_view[:, 1], "bo", label="Detectors")

# Plot rays and angles
for i in range(N_det):
    ax.plot([S_view[0], D_view[i, 0]], [S_view[1], D_view[i, 1]], "k--", alpha=0.6)
    # Optional: annotate angles
    midpoint = (S_view + D_view[i]) / 2
    angle_deg = np.degrees(beta[i])
    ax.text(midpoint[0], midpoint[1], f"{angle_deg:.1f}°", fontsize=8, ha="center")

# Aesthetics
ax.set_aspect("equal")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.legend()
ax.set_title("Fan Beam Projection Geometry")
plt.grid(True)
plt.show()

In [ ]:
plt.figure()
plt.plot(beta, label="Detector angles (beta)")
plt.title("Detector angle sweep")
plt.xlabel("Detector bin index")
plt.ylabel("Angle (rad)")
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
from src.reconstruct import compute_source_frame_coords

gamma = compute_source_frame_coords(N_pixels, D_so, theta=theta)[
    1
]  # shape: [N_pixels, N_pixels, N_views]

plt.figure()
plt.plot(beta, label="Detector beta (rad)")
plt.title("Beta - detector angle values")
plt.xlabel("Detector index")
plt.ylabel("Angle (rad)")
plt.legend()
plt.grid(True)
plt.show()

plt.figure()
plt.imshow(gamma[:, :, 0], cmap="twilight", origin="lower")
plt.title("Gamma angle map for first view")
plt.colorbar(label="Gamma (rad)")
plt.show()